In [ ]:
import pandas as pd
import pvlib 

from bokeh.io import output_notebook
output_notebook()

import captest as pvc

%load_ext autoreload
%autoreload 2

In [ ]:
pvsyst = pvc.CapData('pvsyst')

In [ ]:
pvsyst.load_data(load_pvsyst=True)

In [ ]:
pvsyst.trans

In [ ]:
pvsyst.set_reg_trans(power='real_pwr--', poa='irr-poa-', t_amb='temp-amb-', w_vel='wind--')

In [ ]:
pvsyst.filter_irr(500, 800)

In [ ]:
pvsyst.rep_cond(test_date='4/2/1990')

In [ ]:
pvsyst.reset_flt()

In [ ]:
pvsyst.rep_cond()